# Pointwise
* predict the relevance scores

# Import

In [1]:
from collections import Counter
import numpy as np  
import pandas as pd 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from sklearn.preprocessing import scale 
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# models
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMRanker

In [2]:
RANDOM_STATE = 24

In [3]:
# 導入資料
train_df = pd.read_pickle('/home/adam/Steph_C/my_thesis/data/Train_by_postoal_code_pointwise_v3_3_tfidf.pkl')
test_df = pd.read_pickle('/home/adam/Steph_C/my_thesis/data/Test_by_postoal_code_pointwise_v3_3_tfidf.pkl')

In [4]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [5]:
# # expr. 1 change relevance score to binary
# # change_rel_score = lambda df: 1 if df['relevance']>0 else 0
# def change_rel_score(df):
#     for i in range(len(df)):
#         if df.relevance[i] > 0 :
#             df.relevance[i]=1
#         else:
#             pass
    
#     return df
# train_df = change_rel_score(train_df)
# test_df = change_rel_score(test_df)

In [6]:
train_df = train_df.sample(frac=1 , random_state = RANDOM_STATE).reset_index(drop=True)
test_df = test_df.sample(frac=1 , random_state = RANDOM_STATE).reset_index(drop=True)

In [7]:
len(Counter(train_df.name))

405

In [8]:
train_df.shape

(5984, 46)

In [9]:
test_df.shape

(3160, 46)

# Models

In [10]:
def _precision(predictions , actuals, k = None):
    
    """
    Calculate the precision at k
    
    Returns: a list of precisions
    """
    
    precisions =[]

    for i in range(len(predictions)):
        
        prediction = predictions[i]

        if  k != None:
            prediction =  predictions[i][:k]
        
        score = 0
        for j in prediction:
            if j in actuals[i]:
                score+=1
        if len(prediction) != 0:
            precisions.append(score/len(prediction))
        else:
            precisions.append(0)
    return precisions
    

def _recall(predictions , actuals, k = None):
    
    """
    Calculate the precision at k
    
    Returns: a list of recalls
    """
    recalls =[]
    
    for i in range(len(predictions)):
        
        prediction =  predictions[i]
        
        if  k != None:
            prediction =  predictions[i][:k]
        
        score = 0
        for j in range(len(prediction)):
            if prediction[j] in actuals[i]:
                score+=1
        recalls.append(score/len(actuals[i]))
    
    return recalls



In [11]:
def calculate_mrr(predictions, actuals):
    """
    Calculate the mean reciprocal rank (MRR) for a set of predictions and actual values.
    
    Parameters:
    predictions (list of lists): A list of predicted rankings sorted by probability.
    actual (list of lists): A list of actual rankings sorted by relevance.
    
    Returns:    
    float: A list of MRR scores.
    """
    mrr_list = []
    for i in range(len(predictions)):
        reciprocal_rank = 0
        if actuals[i][0] in predictions[i]:
            reciprocal_rank = 1/ (predictions[i].index(actuals[i][0]) + 1)
        mrr_list.append(reciprocal_rank)
    return mrr_list

In [12]:
def calculate_map( predictions , actuals, k=None):
    """
    Calculate the mean average precision (MAP) for a set of queries.

    Parameters:
    actual (list of sets or lists): A list of sets or lists of the actual relevant items for each query.
    predicted (list of lists): A list of lists of predicted items for each query.
    k (int): The maximum number of predicted items to consider for each query.

    Returns:
    float: A list of MAP scores.
    """
    
    map_list = []
    

    for i in range(len(predictions)):
        
        ap_list = []
        hit = 0 
        cnt = 0 
        
        prediction =  predictions[i]
        
        if k != None:
            prediction =  predictions[i][:k]
        
        
        for j in prediction:
            if j in actuals[i]:
                hit+=1
                cnt+=1
                ap_list.append(hit/cnt)
            else:
                cnt+=1
        if len(ap_list) != 0:
            map_list.append(np.mean(ap_list))
        else:
            map_list.append(0)
    
    return map_list
                

In [13]:
def calculate_dcg_ndcg( predictions , actuals, rel ,k=None):
    """
    Calculate the DCG@k , NDCG@k for a set of queries.

    Parameters:
    actual (list of sets or lists): A list of sets or lists of the actual relevant items for each query.
    predicted (list of lists): A list of lists of predicted items for each query.
    k (int): The maximum number of predicted items to consider for each query.

    Returns:
    float: A list of DCG , NDCG scores.
    """
    dcg_list = []
    ndcg_list = []
    
    for i in range(len(predictions)):
        dcg =0
        idcg =0
        
        prediction = predictions[i]
        
        if k != None:
            prediction = predictions[i][:k]
        
        for j in range(len(actuals[i])):
            if actuals[i][j] in prediction:
                rank = prediction.index(actuals[i][j]) + 1
                dcg += np.divide(float(rel[i][j]),np.log2(rank+1))
            idcg += np.divide(float(rel[i][j]),np.log2((j+1)+1))
        dcg_list.append(dcg)
        if np.divide(dcg,idcg) > 1:
            print(rel[i], prediction,actuals[i]  )
            print(i,dcg,idcg  , 'Wrong !!!')
        ndcg_list.append(np.divide(dcg,idcg))
        
    return dcg_list , ndcg_list

In [14]:
# create list of list for query ranking
def get_ranking(df ):
    
    """
    Turn the probability array into a list of lists for calculation.
    
    Parameters:
    df(DataFrame): the test dataframe
    
    Returns:
    prediction (list of lists): A list of predicted rankings for each query.
    actual (list of lists): A list of actual rankings for each query.
    """
 
    pred_list = []
    pred_rel = []
    true_list = []
    
    
    for res in Counter(df.name):
        
        tmp = df[df.name == res]
        a_sorted = tmp.sort_values(by=['relevance'],ascending=[False])
        p_sorted = tmp.sort_values(by=['predictions'],ascending=[False])
        p_sorted = p_sorted[p_sorted.predictions>0]

        true_list.append(list(a_sorted[a_sorted.relevance!=0].postal_code))
        pred_list.append(list(p_sorted.postal_code))
        pred_rel.append(list(a_sorted[a_sorted.relevance!=0].relevance))
        
        
        
    return pred_list, pred_rel , true_list

# Model

In [15]:
# models
LR = LogisticRegression(random_state=RANDOM_STATE)
RF = RandomForestClassifier(random_state=RANDOM_STATE)
DTC = DecisionTreeClassifier(random_state=RANDOM_STATE)
KNC = KNeighborsClassifier()
SVC = svm.SVC(random_state=RANDOM_STATE)
GNB = GaussianNB()
LGBM = LGBMRanker(objective="lambdarank",random_state=RANDOM_STATE)

# train_features=['density', 'entropy', 'competitiveness','area_pop', 'accessibility','complementary']
# train_features=['density', 'entropy', 'competitiveness','area_pop', 'accessibility','complementary','affinity']
# train_features=['density', 'entropy', 'competitiveness','area_pop', 'accessibility','cosine_sim']
train_features=['density', 'entropy', 'competitiveness','area_pop', 'accessibility','complementary_with_att','affinity_with_att']


get_group_size = lambda df: df.reset_index().groupby("name")['name'].count()

train_groups = get_group_size(train_df).to_numpy()
test_groups = get_group_size(test_df).to_numpy()

print(sum(train_groups) , sum(test_groups))

5984 3160


In [16]:
models = [LR, RF, DTC, KNC, SVC, GNB, LGBM]
model_name =['LR', 'RF', 'DTC', 'KNC', 'SVC', 'GNB','LGBMRanker']
score_dict = {}

for i in range(len(models)):
    score_dict[model_name[i]]={}
    model = models[i]
    # Train
    if model_name[i] != 'LGBMRanker':
        model.fit(train_df[train_features], train_df[['relevance']])
    else:
        model.fit(train_df[train_features], train_df[['relevance']], group=train_groups)
    
    # Predict
    predict = model.predict(test_df[train_features])
    test_df['predictions'] = predict
    pred_list, pred_rel , true_list = get_ranking(test_df)

    # Evaluation
    mrr_list = calculate_mrr(pred_list , true_list)
    map_list = calculate_map(pred_list , true_list)
    dcg_list , ndcg_list = calculate_dcg_ndcg(pred_list , true_list,pred_rel)
    precision_list_1 = _precision(pred_list , true_list, k=1)
    recall_list_1 = _recall(pred_list , true_list,k=1)
    precision_list_3 = _precision(pred_list , true_list, k=3)
    recall_list_3 = _recall(pred_list , true_list,k=3)
    precision_list = _precision(pred_list , true_list)
    recall_list = _recall(pred_list , true_list)

    score_dict[model_name[i]]['precision @ 1'] = np.mean(precision_list_1)
#     score_dict[model_name[i]]['recall @ 1'] = np.mean(recall_list_1)
    score_dict[model_name[i]]['precision @ 3 '] = np.mean(precision_list_3)
    score_dict[model_name[i]]['recall @ 3'] = np.mean(recall_list_3)
    score_dict[model_name[i]]['precision'] = np.mean(precision_list)
    score_dict[model_name[i]]['recall'] = np.mean(recall_list)
    score_dict[model_name[i]]['mrr'] = np.mean(mrr_list)
    score_dict[model_name[i]]['map'] = np.mean(map_list)
    score_dict[model_name[i]]['dcg'] = np.mean(dcg_list)
    score_dict[model_name[i]]['ndcg'] = np.mean(ndcg_list)

/home/adam/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/adam/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,)

# Evaluation

In [17]:
pd.DataFrame(score_dict).round(3)

,LR,RF,DTC,KNC,SVC,GNB,LGBMRanker
precision @ 1,0.104,0.491,0.558,0.380,0.067,0.343,0.538
precision @ 3,0.104,0.490,0.490,0.389,0.065,0.343,0.464
recall @ 3,0.066,0.576,0.680,0.409,0.043,0.275,0.621
precision,0.104,0.490,0.459,0.400,0.065,0.346,0.471
recall,0.066,0.651,0.792,0.497,0.043,0.307,0.765
mrr,0.057,0.444,0.526,0.324,0.036,0.230,0.492
map,0.107,0.603,0.674,0.482,0.068,0.380,0.649
dcg,1.060,8.226,9.673,6.182,0.648,4.476,9.184
ndcg,0.071,0.553,0.662,0.420,0.046,0.293,0.631
